In [1]:
import importlib
import pandas as pd
import xarray as xr
import numpy as np
import sys
import warnings
from scipy import signal 
import glob as glob

from ecpaper_utils import bootstrap_utils as boot
from ecpaper_utils import filter_utils as filt
from ecpaper_utils import averaging_utils as avg

importlib.reload(boot)

warnings.filterwarnings('ignore')

In [2]:
cmip5models=pd.read_csv("../../DATASORT/cmip5csvinfo.csv")
cmip6models=pd.read_csv("../../DATASORT/cmip6csvinfo.csv")

In [3]:
vwind_path="../../DATASORT/VWIND/DATA/"

In [4]:
le_models = [x.split("_")[1].split(".")[0] for x in sorted(glob.glob(vwind_path+"/v300*.nc"))] ## parse files to get model names
le_dat = []

for le in le_models:
    print(le)
    ## past data
    ds = xr.open_dataset(vwind_path+"v300_"+le+".nc")
    ds = ds.assign_coords({"Model": le})
    le_dat.append(ds)

CESM1-CAM5
CSIRO-Mk3-6-0
CanESM2
GFDL-CM3
MPI-ESM


In [5]:
le_dat = xr.concat(le_dat, dim = "Model").sel(lon = slice(200, 300), lat = slice(20, 40))

In [6]:
le_dat

<xarray.Dataset>
Dimensions:    (Model: 5, lat: 21, lon: 101, member: 100)
Coordinates:
  * member     (member) int64 0 1 2 3 4 5 6 7 8 9 ... 91 92 93 94 95 96 97 98 99
  * lat        (lat) float64 20.0 21.0 22.0 23.0 24.0 ... 37.0 38.0 39.0 40.0
  * lon        (lon) float64 200.0 201.0 202.0 203.0 ... 297.0 298.0 299.0 300.0
  * Model      (Model) object 'CESM1-CAM5' 'CSIRO-Mk3-6-0' ... 'MPI-ESM'
Data variables:
    vpast_djf  (Model, member, lat, lon) float64 -6.613 -6.372 ... 2.182 2.153
    vfut_djf   (Model, member, lat, lon) float64 -6.191 -5.98 ... 1.868 1.734

### calculate LE mean $\Delta V_{SW}$ and $|\psi|$

In [7]:
le_vdiff = []
le_vamp = []

for le in le_models: 
    print(le)
    ds = le_dat.sel(Model = le).dropna(dim = "member").mean(dim = "member")
    vpast_eddy = ds["vpast_djf"] - ds["vpast_djf"].mean("lon")
    vfut_eddy = ds["vfut_djf"] - ds["vfut_djf"].mean("lon")
    vpastsw = avg.cosweightlonlat(vpast_eddy, 250, 265, 25, 37)
    vfutsw = avg.cosweightlonlat(vfut_eddy, 250, 265, 25, 37)
        
    le_vdiff.append((vfutsw - vpastsw).values)
    
    # stationary wave ampltiude index
    vpast_k4to32 = filt.filterk(ds["vpast_djf"], 4, 32)
    vpast_amp = np.sqrt(avg.cosweightlonlat(vpast_k4to32**2, 200., 300., 20., 40.))
    
    le_vamp.append(vpast_amp.values)

CESM1-CAM5
CSIRO-Mk3-6-0
CanESM2
GFDL-CM3
MPI-ESM


### calculate errors for CMIP5 models

In [8]:
models = cmip5models['Model']
nmodels = models.shape[0]
nboots = 1000

sig_vdiff = np.zeros(nmodels) 
sig_psi = np.zeros(nmodels) 

for i, model in enumerate(models):
    nmemp=cmip5models.loc[i, "Nmempast"]
    nmemf=cmip5models.loc[i, "Nmemfuture"]
    
    boots_vdiff = []
    boots_vamp = []
    
    for j, le in enumerate(le_models):
        
        ## bootstrapped mean future CA precip trends 
        vpast = boot.bootgen_multimem(le_dat.sel(Model = le)["vpast_djf"].dropna(dim = "member"), 
                                  nboots, nmemf, seed=3)
        vfut = boot.bootgen_multimem(le_dat.sel(Model = le)["vfut_djf"].dropna(dim = "member"), 
                                  nboots, nmemf, seed=3)
        
        vpast_eddy = vpast - vpast.mean("lon")
        vfut_eddy = vfut - vfut.mean("lon")
        vpastsw = avg.cosweightlonlat(vpast_eddy, 250, 265, 25, 37)
        vfutsw = avg.cosweightlonlat(vfut_eddy, 250, 265, 25, 37)
        
        ## change in v300 SW
        boots_vdiff.append((vfutsw - vpastsw).values - le_vdiff[j])
        
        # stationary wave ampltiude index
        vpast_k4to32 = filt.filterk(vpast, 4, 32)
        vpast_amp = np.sqrt(avg.cosweightlonlat(vpast_k4to32**2, 200., 300., 20., 40.))
        
        boots_vamp.append(vpast_amp.values - le_vamp[j])
    
    boots_vdiff = np.concatenate(boots_vdiff)
    boots_vamp = np.concatenate(boots_vamp)
    
    sig_vdiff[i] = np.std(boots_vdiff)
    sig_psi[i] = np.std(boots_vamp)

In [9]:
error_dat5 = xr.Dataset({'sig_vdiff': xr.DataArray(sig_vdiff, coords=[models], dims=["Model"]), 
                       'sig_psi': xr.DataArray(sig_psi, coords=[models], dims=["Model"])})

error_dat5.to_netcdf("./DATA/errors_cmip5.nc")

In [10]:
error_dat5

<xarray.Dataset>
Dimensions:    (Model: 37)
Coordinates:
  * Model      (Model) object 'ACCESS1-0' 'ACCESS1-3' ... 'NorESM1-ME'
Data variables:
    sig_vdiff  (Model) float64 0.7848 0.7848 0.7848 ... 0.7848 0.7848 0.7848
    sig_psi    (Model) float64 0.06046 0.06046 0.06046 ... 0.06046 0.06046

In [11]:
models = cmip6models['Model']
nmodels = models.shape[0]
nboots = 1000

sig_vdiff = np.zeros(nmodels) 
sig_psi = np.zeros(nmodels) 

for i, model in enumerate(models):
    nmemp=cmip6models.loc[i, "Nmempast"]
    nmemf=cmip6models.loc[i, "Nmemfuture"]
    
    boots_vdiff = []
    boots_vamp = []
    
    for j, le in enumerate(le_models):
        
        ## bootstrapped mean future CA precip trends 
        vpast = boot.bootgen_multimem(le_dat.sel(Model = le)["vpast_djf"].dropna(dim = "member"), 
                                  nboots, nmemf, seed=3)
        vfut = boot.bootgen_multimem(le_dat.sel(Model = le)["vfut_djf"].dropna(dim = "member"), 
                                  nboots, nmemf, seed=3)
        
        vpast_eddy = vpast - vpast.mean("lon")
        vfut_eddy = vfut - vfut.mean("lon")
        vpastsw = avg.cosweightlonlat(vpast_eddy, 250, 265, 25, 37)
        vfutsw = avg.cosweightlonlat(vfut_eddy, 250, 265, 25, 37)
        
        ## change in v300 SW
        boots_vdiff.append((vfutsw - vpastsw).values - le_vdiff[j])
        
        # stationary wave ampltiude index
        vpast_k4to32 = filt.filterk(vpast, 4, 32)
        vpast_amp = np.sqrt(avg.cosweightlonlat(vpast_k4to32**2, 200., 300., 20., 40.))
        
        boots_vamp.append(vpast_amp.values - le_vamp[j])
    
    boots_vdiff = np.concatenate(boots_vdiff)
    boots_vamp = np.concatenate(boots_vamp)
    
    sig_vdiff[i] = np.std(boots_vdiff)
    sig_psi[i] = np.std(boots_vamp)

In [12]:
error_dat6 = xr.Dataset({'sig_vdiff': xr.DataArray(sig_vdiff, coords=[models], dims=["Model"]), 
                       'sig_psi': xr.DataArray(sig_psi, coords=[models], dims=["Model"])})

error_dat6.to_netcdf("./DATA/errors_cmip6.nc")

In [13]:
error_dat6

<xarray.Dataset>
Dimensions:    (Model: 38)
Coordinates:
  * Model      (Model) object 'ACCESS-CM2' 'ACCESS-ESM1-5' ... 'UKESM1-0-LL'
Data variables:
    sig_vdiff  (Model) float64 0.7848 0.4542 0.7848 ... 0.7848 0.7848 0.3529
    sig_psi    (Model) float64 0.06046 0.03516 0.06046 ... 0.06046 0.02705